In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("/content/df_one_hot_encoded_normalized.csv")
train_data = df[(df['Target']== '0') | (df['Target']== '1')] # Training data (rows labeled 0 and 1)

X = train_data.drop(columns=['Target'])
y = train_data['Target'].astype(int)

print("Training data shape:", X.shape)

Training data shape: (3630, 252)


In [2]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define base models
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
xgb_model = XGBClassifier(n_estimators=200, learning_rate=0.1, random_state=42, use_label_encoder=False, eval_metric='logloss')
mlp_model = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=500, random_state=42)

# Initialize the Voting Classifier with 'soft' voting
voting_model = VotingClassifier(estimators=[
    ('random_forest', rf_model),
    ('xgboost', xgb_model),
    ('mlp', mlp_model)
], voting='soft')

# Train the Voting Classifier
voting_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = voting_model.predict(X_test)

# Evaluate the model
print("Voting Classifier Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:19:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Voting Classifier Accuracy: 0.9022038567493113

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.95      0.92       449
           1       0.91      0.83      0.87       277

    accuracy                           0.90       726
   macro avg       0.90      0.89      0.89       726
weighted avg       0.90      0.90      0.90       726



In [3]:
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(voting_model, X, y, cv=10, scoring='accuracy')

# Print the results
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())
print("Standard Deviation:", cv_scores.std())

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:20:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:20:54] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:21:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:21:49] WARNING: /workspace/src/learner.cc:740: 
Parameter

Cross-Validation Scores: [0.89807163 0.91184573 0.8953168  0.90082645 0.92286501 0.88980716
 0.87878788 0.93663912 0.90358127 0.91460055]
Mean Accuracy: 0.9052341597796143
Standard Deviation: 0.015930393713279298
